<a href="https://colab.research.google.com/github/iomichiamofede/iomichiamofede/blob/main/IN_CONCERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IN CONCERT structure determination

This Colab notebook allows you to prepare analyze data for the INtegrated COevoultion Nmr Cryo-Em pRoTein structure determination method by [Napoli et. al](https://doi.org/). 

**Requirements**

The method uses chemical shift assignments, distance and dihedral angle restraints from Nuclear Magnetic Resonance, an electron density map from cryo-electron microscopy and evolutionary couplings extracted from Multiple Sequence Alignements.

The user should have access to CS-Rosetta and NAMDINATOR

In [ ]:
#@title Import and install modules

import pandas as pd
import sys
from io import StringIO
import numpy as np
!pip install prody
import glob # Package for Unix-style pathname pattern expansion
import os   # Python operating system interface

In [ ]:
#@title Mount drive

#@markdown <font color='red'> **You have now access to your drive! Do not type anything riskful for your data (i.e. rm *)**

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!pwd

Mounted at /content/drive
/content


In [ ]:
#@title Define functions

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left to define the functions that you will use
#@markdown in this Colab notebook.

#@markdown **Note**: Some functions are adapted from the CS-Rosetta
#@markdown toolbox available [here](https://csrosetta.chemistry.ucsc.edu/downloads/toolbox).

def cyana_2_atompair(lol,upl):

    #### https://csrosetta.bmrb.io/format_help

    #### AtomPair HB3  153 HA   153 BOUNDED 1.700 3.300 .500 NOE
    #The first column should always be "AtomPair".
    #The second column is the atom name of the first atom.
    #The third column is the residue number of the first atom.
    #The fourth column is the atom name of the second atom.
    #The fifth column is the atom name of the second atom.
    #The sixth column is the Rosetta function type. You can use any of the Rosetta functions, but the simplest to use is BOUNDED.
    #The seventh column is the distance lower bound.
    #The eighth column is the distance upper bound.
    #The ninth column should always be .5
    #The tenth column should always be NOE
    print("Running...")
    # lol/upl file looks like this:
    #   7 VAL QG2    185 ILE QD1    3.0 
    colnameslol=['resnum1','resname1','atom1','resnum2','resname2','atom2','lol']
    colnamesupl=['resnum1','resname1','atom1','resnum2','resname2','atom2','upl']
    dflol = pd.read_csv(lol,delim_whitespace=True,names=colnameslol)
    dfupl = pd.read_csv(upl,delim_whitespace=True,names=colnamesupl)
    merged=pd.concat([dflol,dfupl['upl']],axis=1)

    w=''
    #### AtomPair HB3  153 HA   153 BOUNDED 1.700 3.300 .500 NOE
    for index,row in merged.iterrows():
      w+=('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % ('AtomPair',row['atom1'],row['resnum1'],row['atom2'],row['resnum2'],'BOUNDED',row['lol'],row['upl'],0.500,'NOE'))

    out=open('AtomPair_%s.cst' % lol.split('.lol')[0],'w')
    out.write(w)

def renumber_and_trim_atompair(cst,offset,start,end):

  colnames=['AtomPair','atom1','resnum1','atom2','resnum2','BOUNDED','lol','upl','0.500','NOE']
  dfres=pd.read_csv(cst, header = None, delim_whitespace=True, names=colnames)

  w=''
  for index, row in dfres.iterrows():
      if ((int(row['resnum1'])-offset)>start) and ((int(row['resnum2'])-offset)>start) and ((int(row['resnum1'])-offset)<end) and ((int(row['resnum2'])-offset)<end):
          w += ('%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n' % ('AtomPair',row['atom1'],int(row['resnum1'])-offset,row['atom2'],int(row['resnum2'])-offset,'BOUNDED',row['lol'],row['upl'],0.500,'NOE'))
  out_ren=open("%s_ren_by_%s.cst" % (cst.split('.cst')[0],offset),'w')   
  out_ren.write(w)
  out_ren.close()

def DCA_2_cyana(DCA_file):

  # This script converts a DCA .dat file into a cyana restraints.lol file
  # It needs the .dat file
  # .dat file looks like this:
  #    13      26   |     A18     V32       1
  #    282     341  |    H284    K343       2

  # Output .lol looks like:
  # 343	LYS	CB	346	GLU	CB	2.0
  # 34	ILE	CB	38	LYS	CB	2.0
  print ("Running...")

  

  aa_table_one2three=	{
    "A" : "ALA",
    "C" : "CYS",
    "D" : "ASP",
    "E" : "GLU",
    "F" : "PHE",
    "G" : "GLY",
    "H" : "HIS",
    "I" : "ILE",
    "K" : "LYS",
    "L" : "LEU",
    "M" : "MET",
    "N" : "ASN",
    "P" : "PRO",
    "Q" : "GLN",
    "R" : "ARG",
    "S" : "SER",
    "T" : "THR",
    "V" : "VAL",
    "W" : "TRP",
    "Y" : "TYR"}

  # Read the input file
  dfprot = pd.read_csv(DCA_file,delim_whitespace=True,header = None)

  w=''
  v=''
  for index, row in dfprot.iterrows():
      res1nam=aa_table_one2three[row[3][0]]
      res1num=row[3][1:]
      res2nam=aa_table_one2three[row[4][0]]
      res2num=row[4][1:]
      atom1='CB'
      atom2='CB'
      if res1nam=='GLY': 
          atom1='CA'
      if res2nam=='GLY': 
          atom2='CA'  
      w+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (res1num,res1nam,atom1,res2num,res2nam,atom2,2.0))
      v+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (res1num,res1nam,atom1,res2num,res2nam,atom2,7.0))

  outL=open('Unfiltered_%s.lol' % DCA_file.split('.dat')[0],'w')
  outU=open('Unfiltered_%s.upl' % DCA_file.split('.dat')[0],'w')
  outL.write(w)
  outU.write(v)

def Network_Anchoring(coev_file,NMR_file):

  ######################################
  # Parameters to be tweaked: (could be made as input parameters rather than hard-coded here!)
  upl=7.0 # The distance limit that will be written in the output file
  seqtolerance = 4 # it will search for contacts around the given residue plus/minus this number

  class restraint:
    def __init__(self, res1, aa1,atom1,res2, aa2,atom2,source):
      if res1<res2:
          self.res1 = res1
          self.res2 = res2
          self.aa1 = aa1
          self.aa2 = aa2
          self.atom1 = atom1
          self.atom2 = atom2
      else:
          self.res1 = res2
          self.res2 = res1
          self.aa1 = aa2
          self.aa2 = aa1
          self.atom1 = atom2
          self.atom2 = atom1

      self.source = source

  columnnames=['res1','aa1','atom1','res2','aa2','atom2','dist']

  restraintlist1=[] # This will collect all the restraints from the first set of files (e.g. coevolution)
  restraintlist2=[] # This will collect all the restraints from the second set of files (e.g. NMR)

  a=pd.read_csv(coev_file,delim_whitespace=True,header=None,names=columnnames)
  source = coev_file
  for index, row in a.iterrows():
          restraintlist1.append(restraint(row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],source))

  b=pd.read_csv(NMR_file,delim_whitespace=True,header=None,names=columnnames)
  source = NMR_file
  for index, row in b.iterrows():
      restraintlist2.append(restraint(row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],source))


  outscore0=open('Discarded_%s.upl' % (coev_file.split('.lol')[0]).split('Unfiltered_')[1],'w')
  outscore1=open('Filtered_%s.upl' % (coev_file.split('.lol')[0]).split('Unfiltered_')[1],'w')
  outscore1L=open('Filtered_%s.lol' % (coev_file.split('.lol')[0]).split('Unfiltered_')[1],'w')
  outdetails=open('Network_Ancoring_details_%s' % (coev_file.split('.lol')[0]).split('Unfiltered_')[1],'w')

  restraintsdata1=[]
  restraintsdata2=[]

  for q in range(len(restraintlist1)):
      restraintsdata1.append([restraintlist1[q].res1,restraintlist1[q].aa1,restraintlist1[q].atom1,restraintlist1[q].res2,restraintlist1[q].aa2,restraintlist1[q].atom2,restraintlist1[q].source])
      
  for p in range(len(restraintlist2)):
      restraintsdata2.append([restraintlist2[p].res1,restraintlist2[p].aa1,restraintlist2[p].atom1,restraintlist2[p].res2,restraintlist2[p].aa2,restraintlist2[p].atom2,restraintlist2[p].source])
      
  df_restraints1=pd.DataFrame.drop_duplicates(pd.DataFrame(restraintsdata1,columns=['res1','aa1','atom1','res2','aa2','atom2','source'])) # Add more columns here    
  df_restraints2=pd.DataFrame.drop_duplicates(pd.DataFrame(restraintsdata2,columns=['res1','aa1','atom1','res2','aa2','atom2','source'])) # Add more columns here
  data3=[]
  data2=[]
  data1=[]
  data0=[]
  ############################
  # Now go through all the restraints in list 1 and check if they are there in list 2

  for index, row in df_restraints1.iterrows():
      testtuplelist=[]
      currentrestraint=(row['res1'],row['res2'])
      for tol1 in np.arange(-seqtolerance,seqtolerance+1): # allow the search within some tolerance on residue 1
          for tol2 in np.arange(-seqtolerance,seqtolerance+1): # allow the search within some tolerance on residue 2
              testtuplelist.append((row['res1']+tol1,row['res2']+tol2))
      testflag=0
      supportingrestraints=[]
      #supportingrestraints_full=[]
      outdetails.write("\n") # This is just to have separation lines in the details file; decoration.
      for k, line in df_restraints2.iterrows():
          reftuple=(line['res1'],line['res2'])
          if reftuple in testtuplelist:
              testflag+=1
              supportingrestraints.append(reftuple)
              outdetails.write("Search restraint %s\t%s\t%s\t%s\t%s\t%s\tfrom\t%s\t Found restraint %s\t%s\t%s\t%s\t%s\t%s\tin\t%s\n" % (row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],row['source'],line['res1'],line['aa1'],line['atom1'],line['res2'],line['aa2'],line['atom2'],line['source']))

      if testflag>0:  # found one or more matches
  #		outscore1.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (restraintlist1[i].res1,restraintlist1[i].aa1,restraintlist1[i].atom1,restraintlist1[i].res2,restraintlist1[i].aa2,restraintlist1[i].atom2,upl))
          data1.append([row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],upl])

      else: # constraint not supported
  #		outscore0.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (restraintlist1[i].res1,restraintlist1[i].aa1,restraintlist1[i].atom1,restraintlist1[i].res2,restraintlist1[i].aa2,restraintlist1[i].atom2,upl))
          data0.append([row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],upl])

  columnsfinal=['res1','aa1','atom1','res2','aa2','atom2','upl']
  df1=pd.DataFrame.drop_duplicates(pd.DataFrame(data1, columns=columnsfinal))
  df0=pd.DataFrame.drop_duplicates(pd.DataFrame(data0, columns=columnsfinal))

  for index, row in df1.iterrows():
      outscore1.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],row['upl']))
      outscore1L.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],2.0))
  for index, row in df0.iterrows():
      outscore0.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (row['res1'],row['aa1'],row['atom1'],row['res2'],row['aa2'],row['atom2'],row['upl']))


  outscore1.close()
  outscore1L.close()
  outscore0.close()
  outdetails.close()

def CEYAPP_2_cyana(CEYAPP_file):
  
  # This function converts a contacts.txt (CE-YAPP) format into 2 cyana format constraints files (.lol [2 A] + .upl [7 A])  
  # It needs the .txt file
  # txt file looks like this:
  # 343 LYS CB 346 GLU CB    1.00
  # 34 ILE CB 38 LYS CB      1.00
  # Output .lol looks like:
  #  7   VAL     QG2       185  ILE QD1    3.0 
  #  15  VAL     QG2       16   VAL QG2    3.0 
  # Output .upl looks like:
  #  7   VAL     QG2       185  ILE QD1    15.0 
  #  15  VAL     QG2       16   VAL QG2    15.0 

  print( "Running...")

 # A dictionary for the conversion of atom names
  atom_table=	{
    "HA" : "QA",
    "HB" : "QB",
    "HG" : "QG",
    "HD" : "QD",
    "HG1" : "QG1",
    "HG2" : "QG2",
    "HD1" : "QD1",
    "HD2" : "QD2",
    "CA" : "CA",
    "CB" : "CB",
    "CG" : "CG",
    "CD" : "CD",
    "CG1" : "CG1",
    "CG2" : "CG2",
    "CD1" : "CD1",
    "CD2" : "CD2",
      "N" : "N"}

  conts=CEYAPP_file
 
  # Read the input file
  dfprot = pd.read_csv(conts,delim_whitespace=True,header = None)

  w=''
  for index, row in dfprot.iterrows():
    w+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (row[0],row[1], atom_table[row[2]],row[3],row[4], atom_table[row[5]],2.0))
  out=open('Unfiltered_%s.lol' % conts.split('.txt')[0],'w')
  out.write(w)

  w=''
  out=''
  for index, row in dfprot.iterrows():
    w+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (row[0],row[1], atom_table[row[2]],row[3],row[4], atom_table[row[5]],7.0))

  out=open('Unfiltered_%s.upl' % conts.split('.txt')[0],'w')
  out.write(w)

def GREMLIN_2_cyana(GREMLIN_file):
  # This script converts a GREMLIN file into a cyana restraints.lol file
  # It needs the GREMLIN file
  # GREMLIN file looks like this:
  #K 343	E 346	4.338	1.00	3.30

  # Output .lol looks like:
  # 343	LYS	CB	346	GLU	CB	2.0
  # 34	ILE	CB	38	LYS	CB	2.0

  print ("Running...")

  aa_table_one2three=	{
    "A" : "ALA",
    "C" : "CYS",
    "D" : "ASP",
    "E" : "GLU",
    "F" : "PHE",
    "G" : "GLY",
    "H" : "HIS",
    "I" : "ILE",
    "K" : "LYS",
    "L" : "LEU",
    "M" : "MET",
    "N" : "ASN",
    "P" : "PRO",
    "Q" : "GLN",
    "R" : "ARG",
    "S" : "SER",
    "T" : "THR",
    "V" : "VAL",
    "W" : "TRP",
    "Y" : "TYR"}

  dat=GREMLIN_file
  # Read the input file
  dfprot = pd.read_csv(dat,delim_whitespace=True,header = None)

  w=''
  v=''
  for index, row in dfprot.iterrows():
      res1nam=aa_table_one2three[row[0]] 
      res1num=row[1]
      res2nam=aa_table_one2three[row[2]]
      res2num=row[3]
      atom1='CB'
      atom2='CB'
      if res1nam=='GLY': 
          atom1='CA'
      if res2nam=='GLY': 
          atom2='CA'  
      w+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (res1num,res1nam,atom1,res2num,res2nam,atom2,2.0))
      v+= ("%s\t%s\t%s\t%s\t%s\t%s\t%1.1f\n" % (res1num,res1nam,atom1,res2num,res2nam,atom2,7.0))

  outL=open('Unfiltered_%s.lol' % dat.split('.dat')[0],'w')
  outU=open('Unfiltered_%s.upl' % dat.split('.dat')[0],'w')
  outL.write(w)
  outU.write(v)

In [ ]:
#@markdown ---
#@markdown ### <font color='yellow'> 📁 Enter your files path: 
file_path = "drive/MyDrive/IN_CONCERT" #@param {type:"string"}
%cd $file_path
#@markdown ### <font color='yellow'> Enter your files names:
NMRlow='NMR.lol' #@param {type:"string"}
NMRup='NMR.upl' #@param {type:"string"}
COEV_file='GREMLIN_contacts.dat' #@param {type:"string"}
#@markdown ### <font color='yellow'> Enter your CoEvolution data type:
CoEvolution_Type = 'GREMLIN' #@param ["DCA", "GREMLIN", "CEYAPP"]
#@markdown ---

/content/drive/MyDrive/IN_CONCERT


In [ ]:
#@title Network Anchoring
if(CoEvolution_Type=="DCA"):
  DCA_2_cyana(COEV_file)
  COEV_unfiltered_low='Unfiltered_%s.lol' % COEV_file.split('.dat')[0]
  COEV_unfiltered_up='Unfiltered_%s.upl' % COEV_file.split('.dat')[0]
  Network_Anchoring(COEV_unfiltered_low,NMRlow)
  COEV_filtered_low='Filtered_%s.lol' % COEV_file.split('.dat')[0]
  COEV_filtered_up='Filtered_%s.upl' % COEV_file.split('.dat')[0]
elif(CoEvolution_Type=="GREMLIN"):
  GREMLIN_2_cyana(COEV_file)
  COEV_unfiltered_low='Unfiltered_%s.lol' % COEV_file.split('.dat')[0]
  COEV_unfiltered_up='Unfiltered_%s.upl' % COEV_file.split('.dat')[0]
  Network_Anchoring(COEV_unfiltered_low,NMRlow)
  COEV_filtered_low='Filtered_%s.lol' % COEV_file.split('.dat')[0]
  COEV_filtered_up='Filtered_%s.upl' % COEV_file.split('.dat')[0]
else:
  CEYAPP_2_cyana(COEV_file)
  COEV_unfiltered_low='Unfiltered_%s.lol' % COEV_file.split('.txt')[0]
  COEV_unfiltered_up='Unfiltered_%s.upl' % COEV_file.split('.txt')[0]
  Network_Anchoring(COEV_unfiltered_low,NMRlow)
  COEV_filtered_low='Filtered_%s.lol' % COEV_file.split('.txt')[0]
  COEV_filtered_up='Filtered_%s.upl' % COEV_file.split('.txt')[0]

Running...


In [ ]:
#@title Convert restraints to Rosetta format
cyana_2_atompair(NMRlow,NMRup)
NMRcst=('AtomPair_%s.cst' % NMRlow.split('.lol')[0])
cyana_2_atompair(COEV_filtered_low,COEV_filtered_up)
COEVcst=('AtomPair_%s.cst' % COEV_filtered_low.split('.lol')[0])


Running...
Running...


In [ ]:
#@title Renumber and trim
#@markdown ### <font color='yellow'> Enter your settings:
start_position = 5 #@param {type:"integer"}
end_position = 335 #@param {type:"integer"}
offset = 12 #@param {type:"slider", min:-50, max:50, step:1}
renumber_and_trim_atompair(NMRcst,offset,start_position,end_position)
renumber_and_trim_atompair(COEVcst,offset,start_position,end_position)